In [ ]:
from __future__ import print_function, division

import pandas as pd
import numpy as np 
from pprint import pprint
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import pickle
import scipy
import patsy
import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge 
from sklearn.linear_model import Lasso, ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error, r2_score, make_scorer
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import normaltest


%matplotlib inline 

### Read Data 

In [10]:
with open('../../Data/movies_clean.pickle', 'rb') as f:
    movies_df = pickle.load(f)

In [ ]:
# Plot data 
plt.figure(figsize=(10, 6))
plt.hist(movies_df_2D.DomesticTotalGross,
         bins=10,
         alpha=0.8,
         color='k')
plt.xlabel('2D Movie Total Domestic Gross', fontsize=15)
plt.ylabel('Movie Counts', fontsize=15)
plt.title('2D Movies Total Domestic Gross Distribution')

# plt.savefig('../../Images/model_2d hist.png', 
#             dpi=200, bbox_inches = 'tight')

### Dataset Standardization - removes the mean and scaling to unit variance 
L1 and L2 regularizers of linear models assume that all features are centered around 0 and have variance in the same order. If a feature has a variance that is orders of magnitude larger that others, it might dominate the objective function and make the estimator unable to learn from other features correctly as expected.

In [ ]:
ssX = StandardScaler()
not_Y = movies_df_2D[[x for x in 
                      movies_df_2D.columns if x != 'DomesticTotalGross']] 
X = not_Y[[x for x in not_y.columns if x != 'Movie Title']]
X = X[[x for x in X.columns if x != 'Genre_3D']]
Y = movies_df_2D.DomesticTotalGross

#### Train-Test Split 

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(
    X, Y, test_size=0.1, random_state=42)

X_tr = ssX.fit_transform(X_train)
X_val = ssX.transform(X_val)

### Linear Regression Model 1: SkLearn 

In [27]:
def sklearn_lr(feature,cv = 5, scoring = 'neg_mean_absolute_error'):
    x_train = df_train[feature]
    lr = LinearRegression()
    shape = len(feature)
    x_train = x_train.values.reshape(-1,shape)
    lr.fit(x_train, Y_train)
    score_ = cross_val_score(lr, x_train, Y_train, 
                             cv = cv, scoring = scoring)
    score_mean = score_.mean()
    intercept_ = lr.intercept_
    coef_ = lr.coef_
    print('|Mean score(Neg MSE)|: ', score_mean
          , '\n', 'Intercept|: ',intercept_, '\n', '|Coefs|: ', coef_)

### Linear Regression Model 2 - Lasso Regression (L1)

In [ ]:
def lasso_modeling(feature, cv = 5, scoring = 'neg_mean_absolute_error'):
    x_train = df_train[feature]
    shape = len(feature)
    x_train = x_train.values.reshape(-1,shape)
    model = Lasso(max_iter=10000)
    parameters = {'alpha': [1e-5,1e-3,1e-1,1], 
                  'fit_intercept': [True,False]}
    grid = GridSearchCV(model,parameters, cv=cv, 
                        scoring=scoring, n_jobs=1)
    grid.fit(x_train, Y_train)
    best_lasso = grid.best_estimator_
    best_score = grid.best_score_
    best_params = grid.best_params_
    best_intercept = best_lasso.intercept_
    best_coefs = best_lasso.coef_
    print('|Best Lasso|: ',best_lasso
          ,'\n','|Score (Neg MAE)|: ',best_score
          ,'\n','|Intercept|: ',best_intercept
          ,'\n','|Coefficients|: ',best_coefs)
    return best_lasso

### Linear Regression Model 3 - Ridge Regression (L2)

In [34]:
def ridge_modeling(feature, cv = 5, 
                   scoring = 'neg_mean_absolute_error'):
    x_train = df_train[feature]
    shape = len(feature)
    x_train = x_train.values.reshape(-1,shape)
    model = Ridge(max_iter=5000)
    parameters = {'alpha': [1e-5,1e-3,1e-1,1], 
                  'fit_intercept': [True,False]}
    grid = GridSearchCV(model,parameters, cv=cv, 
                        scoring=scoring, n_jobs=1)
    grid.fit(x_train, Y_train)
    best_ridge = grid.best_estimator_
    best_score = grid.best_score_
    best_params = grid.best_params_
    best_intercept = best_ridge.intercept_
    best_coefs = best_ridge.coef_
    print('|Best Ridge|: ',best_ridge
          ,'\n','|Score (Neg MAE)|: ',best_score
          ,'\n','|Intercept|: ',best_intercept
          ,'\n','|Coefficients|: ',best_coefs)

### Linear Regression Model 4 - ElasticNet

In [37]:
def elasticnet_modeling(feature, cv = 5, 
                        scoring = 'neg_mean_absolute_error'):
    x_train = df_train[feature]
    shape = len(feature)
    x_train = x_train.values.reshape(-1,shape)
    model = ElasticNet(max_iter=5000)
    parameters = {'alpha': [1e-5,1e-3,1e-1,1], 
                  'fit_intercept': [True,False]}
    grid = GridSearchCV(model,parameters, cv=cv,
                        scoring=scoring, n_jobs=1)
    grid.fit(x_train, Y_train)
    best_en = grid.best_estimator_
    best_score = grid.best_score_
    best_params = grid.best_params_
    best_intercept = best_en.intercept_
    best_coefs = best_en.coef_
    print('|Best ElesticNet|: ',best_en
          ,'\n','|Score (Neg MAS)|: ',best_score
          ,'\n','|Intercept|: ',best_intercept
          ,'\n','|Coefficients|: ',best_coefs)

### Result

In [ ]:
preds = model.predict(X_val)
for true,pred in zip(Y_val, preds):
    resid = true - pred
    print("pred, resid:", str(pred) + ", $"+ str(resid))

#### Actual VS Predicted Plot

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(lasso_pred,Y_val,alpha=.8,color='k')
plt.plot(np.linspace(0,200,500),np.linspace(0,200,500),color='red')
plt.xlabel('Predicted Movie Domestic Total Gross', fontsize=15)
plt.ylabel('Actual Movie Domestic Total Gross', fontsize=15)
plt.title('Model[2D] Performance', fontsize=20)

# plt.savefig('../../Images/model_2d performance.png', 
#             dpi=200, bbox_inches = 'tight')

#### Residual Plot 

In [ ]:
def residual_scatter(model, x, y, color):
    lasso_pred = model.predict(x)
    residual_ = []
    pred_ = []
    for true,pred in zip(y, lasso_pred):
        resid = true - pred
        residual_.append(resid)
        pred_.append(pred)
    residual_scatter = plt.scatter(pred_, residual_, color = color)
    plt.ylim(75,-75)

#### Metrics 

In [ ]:
def adj_r2(rsquare, num_data, num_features):
    temp = (1-rsquare)*(num_data-1)
    temp = temp/(num_data-num_features-1)
    temp = 1 - temp
    return temp

In [ ]:
def residual_hist(model, x, y, color, outlier_value=260):
    lasso_pred = model.predict(x)
    residual_ = []
    for true,pred in zip(y, lasso_pred):
        resid = true - pred
        residual_.append(resid)
        residual_ = [i for i in residual_ if abs(i) < outlier_value]
    residual_hist = plt.hist(residual_, color = color, bins=30)
    return normaltest(residual_)

In [ ]:
def standard_error_estimate(true,pred,num_data):
    sse = 0
    for y,ypred in zip(true,pred):
        sse += (y-ypred)**2
    return np.sqrt(sse/(num_data-2))

In [ ]:
def test_model_results(true, X, pred):
    print("Mean Squared Error: ", 
          mean_squared_error(true,pred))
    print("Root Mean Squared Error: ", 
          np.sqrt(mean_squared_error(true,pred)))
    print("Mean Absolute Error: ",
          mean_absolute_error(true,pred))
    
    r2 = r2_score(true,pred)
    print("R2: ", r2)
    print("Adj R2: ", adj_r2(r2,X.shape[0],X.shape[1]))
    print("Standard Error of Estimate: ", 
          standard_error_estimate(true,pred,X.shape[0]))